In [1]:
import glob
import pandas as pd
import numpy as np

import dosemetrics.dvh as dvh

In [2]:
def get_volumes(file_name):
    volumes = {}
        
    df = pd.DataFrame()
    with open(file_name, 'r') as f:
        for line in f:
            if "Structure:" in line:
                idx = line.find(" ") + 1
                struct = line[idx:]
                name = struct.split("\n")[0]
                #print("parsing: " + name)
                for line in f:
                    if 'Volume [cm' in line:
                        idy = line.find(":") + 2
                        vol = line[idy:]
                        volume = vol.split("\n")[0]
                        #print(name + ": " + volume)
                        volumes[name] = [volume]
                        break
    return volumes


# Determine Area Between the Curves ABC.
def abc_value(all_dvhs, directory):
    ABC_values = {}
    for structure in all_dvhs.columns.values:
        if structure.endswith("vol"):
            difference = abs(all_dvhs.loc["AS_plan"][structure] - all_dvhs.loc["Ref_plan"][structure])
            abc_fraction = np.nansum(difference) / 100000
            name = structure.split("\n")[0]
            if name.startswith("R_"):
                name = name.split("R_")[-1]
            ABC_values[name] = abc_fraction
    case_name = directory.split("\\")[-1]         
    ABC_df = pd.DataFrame(ABC_values, index=[case_name]).T
    return ABC_df


In [3]:
####### Main #######
# select all cases in the data folder and iteratively determine all the DVHs
directory = "/Users/amithkamath/data/DLDP/astute-dvh"
cases = glob.glob(directory + "/070/")
ABC_df = pd.DataFrame()
for case in cases:
    case_name = case.split("\\")[-1]
    # select the folder for which you want DVH curves. Select all the DVH files from the folder
    dvh_files = glob.glob(case + "/*.txt")
    n = len(dvh_files)
    print(dvh_files)

    # Retrieve the name of the study from the file names.
    names = []
    for name in dvh_files:
        names.append(name.split("\\")[-1].split(".")[0])
    print(names)

    # Reads all relevant data from the text files and constructs a concatenated dataframe of all dvh files 
    # in the selected folder
    dvhs = []
    for file in dvh_files:
        print("parsing" + file)
        dvhs.append(dvh.read_dvh_file(file))
    all_dvhs = pd.concat(dvhs, keys=names)
    

['/Users/amithkamath/data/DLDP/astute-dvh/070/070_Alt2.txt', '/Users/amithkamath/data/DLDP/astute-dvh/070/070_Alt3.txt', '/Users/amithkamath/data/DLDP/astute-dvh/070/070_Alt1.txt', '/Users/amithkamath/data/DLDP/astute-dvh/070/070_Alt4.txt', '/Users/amithkamath/data/DLDP/astute-dvh/070/070_Reference.txt']
['/Users/amithkamath/data/DLDP/astute-dvh/070/070_Alt2', '/Users/amithkamath/data/DLDP/astute-dvh/070/070_Alt3', '/Users/amithkamath/data/DLDP/astute-dvh/070/070_Alt1', '/Users/amithkamath/data/DLDP/astute-dvh/070/070_Alt4', '/Users/amithkamath/data/DLDP/astute-dvh/070/070_Reference']
parsing/Users/amithkamath/data/DLDP/astute-dvh/070/070_Alt2.txt
parsing/Users/amithkamath/data/DLDP/astute-dvh/070/070_Alt3.txt
parsing/Users/amithkamath/data/DLDP/astute-dvh/070/070_Alt1.txt
parsing/Users/amithkamath/data/DLDP/astute-dvh/070/070_Alt4.txt
parsing/Users/amithkamath/data/DLDP/astute-dvh/070/070_Reference.txt


KeyError: 'AS_plan'